### Image Processing Workflow

1. **Process each image** and obtain a prediction from the SageMaker endpoint.
2. **If the prediction indicates** that the image contains a bird, the image’s file path and prediction will be written to a CSV file.
3. **Final Output**: The resulting CSV file will list only the images that contain birds, which can then be used for batch processing.

This approach efficiently categorizes images, enabling streamlined post-processing of bird images.


In [ ]:
import boto3
import sagemaker
from sagemaker.predictor import Predictor
from sagemaker.serializers import IdentitySerializer
from sagemaker.deserializers import JSONDeserializer
from PIL import Image
from io import BytesIO
import os
import csv

# Initialize SageMaker session and S3 client
sagemaker_session = sagemaker.Session()
s3_client = boto3.client('s3')

# Set your parameters
model_endpoint_name = 'pytorch-inference-2024-11-10-14-10-10-654'
input_bucket = 'csml-data-bucket'
input_folder = 'Photos_to_classify_for_Ryan/'
output_csv = 'bird_predictions.csv'

# Define class names and mapping
class_names = ['birds', 'no_birds']
class_to_index = {'birds': 0, 'no_birds': 1}

# SageMaker Predictor setup
predictor = Predictor(
    endpoint_name=model_endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=IdentitySerializer("image/jpeg"),
    deserializer=JSONDeserializer()
)

def resize_and_pad_image(image, target_size=224):
    """Resize and pad an image to the target size."""
    original_width, original_height = image.size
    aspect_ratio = original_width / original_height
    if original_width > original_height:
        new_width = target_size
        new_height = int(target_size / aspect_ratio)
    else:
        new_height = target_size
        new_width = int(target_size * aspect_ratio)
    img_resized = image.resize((new_width, new_height), Image.LANCZOS)
    new_img = Image.new("RGB", (target_size, target_size), (255, 255, 255))
    paste_x = (target_size - new_width) // 2
    paste_y = (target_size - new_height) // 2
    new_img.paste(img_resized, (paste_x, paste_y))
    return new_img

def classify_image(s3_key):
    """Classify an image and return the result."""
    
    # Download the image from S3
    download_path = '/tmp/' + os.path.basename(s3_key)
    s3_client.download_file(input_bucket, s3_key, download_path)
    
    # Load the image and preprocess it for classification
    with open(download_path, 'rb') as image_file:
        img = Image.open(image_file).convert('RGB')
    img_preprocessed = resize_and_pad_image(img)
    
    # Convert preprocessed image to bytes
    buffer = BytesIO()
    img_preprocessed.save(buffer, format='JPEG')
    payload = buffer.getvalue()

    # Invoke the SageMaker endpoint to classify the image
    response = predictor.predict(payload)
    print(f"Raw prediction response for {s3_key}: {response}")

    # Extract the predicted class and probability
    try:
        predicted_index = int(response['predicted_class'])
        classification_result = class_names[predicted_index]
        probabilities = response['probabilities'][0]  # Assuming response returns probabilities as a list of lists
        bird_probability = probabilities[0]  # Probability for "birds" class (index 0)
    except KeyError:
        print("Error: 'predicted_class' key not found in response.")
        return None, None, s3_key

    return classification_result, bird_probability, s3_key

def process_images_in_s3():
    """Process all images in the input folder, classify them, and save predictions to a CSV."""
    
    # Open the CSV file for writing predictions
    with open(output_csv, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Image S3 Path', 'Prediction', 'Bird Probability'])  # CSV header
        
        # List all objects recursively in the input folder using pagination
        paginator = s3_client.get_paginator('list_objects_v2')
        for page in paginator.paginate(Bucket=input_bucket, Prefix=input_folder):
            if 'Contents' not in page:
                print("No images found in the specified S3 folder.")
                continue

            # Process each image in the current page
            for obj in page['Contents']:
                s3_key = obj['Key']
                if s3_key.endswith(('jpg', 'jpeg', 'png', 'JPG', 'JPEG', 'PNG')):  # Process only images
                    classification_result, bird_probability, image_path = classify_image(s3_key)
                    
                    # Write the full path, classification, and bird probability to CSV
                    if classification_result == 'birds' and bird_probability > 0.89:
                        writer.writerow([image_path, classification_result, bird_probability])
                        print(f"Image {s3_key} classified as {classification_result} with {bird_probability:.2f} probability and added to CSV.")
                        
            
def delete_endpoint(endpoint_name):
    """Delete the specified SageMaker endpoint."""
    try:
        sm_client.delete_endpoint(EndpointName=endpoint_name)
        print(f"SageMaker endpoint '{endpoint_name}' deleted successfully.")
    except Exception as e:
        print(f"Failed to delete endpoint '{endpoint_name}': {e}")


# Run the classification and save results to CSV locally
process_images_in_s3()


# Delete the endpoint after processing
delete_endpoint(model_endpoint_name)

Raw prediction response for Photos_to_classify_for_Ryan/Nest 028_poor view/Nest 028_poor view/2023_0524_162637_001.JPG: {'predicted_class': 1, 'probabilities': [[0.10247521102428436, 0.8975248336791992]]}
Raw prediction response for Photos_to_classify_for_Ryan/Nest 028_poor view/Nest 028_poor view/2023_0524_162641_001.JPG: {'predicted_class': 1, 'probabilities': [[0.30146920680999756, 0.6985307931900024]]}
Raw prediction response for Photos_to_classify_for_Ryan/Nest 028_poor view/Nest 028_poor view/2023_0524_162658_001.JPG: {'predicted_class': 0, 'probabilities': [[0.9951913356781006, 0.004808689467608929]]}
Image Photos_to_classify_for_Ryan/Nest 028_poor view/Nest 028_poor view/2023_0524_162658_001.JPG classified as birds with 1.00 probability and added to CSV.
Raw prediction response for Photos_to_classify_for_Ryan/Nest 028_poor view/Nest 028_poor view/2023_0524_162715_001.JPG: {'predicted_class': 1, 'probabilities': [[0.21459463238716125, 0.7854053378105164]]}
Raw prediction respons

Raw prediction response for Photos_to_classify_for_Ryan/Nest 028_poor view/Nest 028_poor view/2023_0524_163734_001.JPG: {'predicted_class': 1, 'probabilities': [[0.042289428412914276, 0.9577105641365051]]}
Raw prediction response for Photos_to_classify_for_Ryan/Nest 028_poor view/Nest 028_poor view/2023_0524_163750_001.JPG: {'predicted_class': 1, 'probabilities': [[0.05663829296827316, 0.9433616995811462]]}
Raw prediction response for Photos_to_classify_for_Ryan/Nest 028_poor view/Nest 028_poor view/2023_0524_163808_001.JPG: {'predicted_class': 1, 'probabilities': [[0.07600423693656921, 0.9239957332611084]]}
Raw prediction response for Photos_to_classify_for_Ryan/Nest 028_poor view/Nest 028_poor view/2023_0524_163834_001.JPG: {'predicted_class': 1, 'probabilities': [[0.07730806618928909, 0.9226920008659363]]}
Raw prediction response for Photos_to_classify_for_Ryan/Nest 028_poor view/Nest 028_poor view/2023_0524_163907_001.JPG: {'predicted_class': 0, 'probabilities': [[0.889896810054779

Raw prediction response for Photos_to_classify_for_Ryan/Nest 028_poor view/Nest 028_poor view/2023_0524_165524_001.JPG: {'predicted_class': 1, 'probabilities': [[0.00471227103844285, 0.9952876567840576]]}
Raw prediction response for Photos_to_classify_for_Ryan/Nest 028_poor view/Nest 028_poor view/2023_0524_165550_001.JPG: {'predicted_class': 1, 'probabilities': [[0.00715000880882144, 0.9928499460220337]]}
Raw prediction response for Photos_to_classify_for_Ryan/Nest 028_poor view/Nest 028_poor view/2023_0524_165611_001.JPG: {'predicted_class': 1, 'probabilities': [[0.009898877702653408, 0.9901010990142822]]}
Raw prediction response for Photos_to_classify_for_Ryan/Nest 028_poor view/Nest 028_poor view/2023_0524_165627_001.JPG: {'predicted_class': 1, 'probabilities': [[0.006297016981989145, 0.9937029480934143]]}
Raw prediction response for Photos_to_classify_for_Ryan/Nest 028_poor view/Nest 028_poor view/2023_0524_165644_001.JPG: {'predicted_class': 1, 'probabilities': [[0.00529104936867

Raw prediction response for Photos_to_classify_for_Ryan/Nest 028_poor view/Nest 028_poor view/2023_0524_174145_001.JPG: {'predicted_class': 1, 'probabilities': [[0.007523659151047468, 0.9924764037132263]]}
Raw prediction response for Photos_to_classify_for_Ryan/Nest 028_poor view/Nest 028_poor view/2023_0524_174207_001.JPG: {'predicted_class': 0, 'probabilities': [[0.8215053677558899, 0.1784946322441101]]}
Raw prediction response for Photos_to_classify_for_Ryan/Nest 028_poor view/Nest 028_poor view/2023_0524_174225_001.JPG: {'predicted_class': 1, 'probabilities': [[0.010227380320429802, 0.989772617816925]]}
Raw prediction response for Photos_to_classify_for_Ryan/Nest 028_poor view/Nest 028_poor view/2023_0524_174316_001.JPG: {'predicted_class': 1, 'probabilities': [[0.007407006341964006, 0.992592990398407]]}
Raw prediction response for Photos_to_classify_for_Ryan/Nest 028_poor view/Nest 028_poor view/2023_0524_174340_001.JPG: {'predicted_class': 1, 'probabilities': [[0.0048671704716980

Raw prediction response for Photos_to_classify_for_Ryan/Nest 028_poor view/Nest 028_poor view/2023_0524_191038_001.JPG: {'predicted_class': 1, 'probabilities': [[0.0130746616050601, 0.9869253635406494]]}
Raw prediction response for Photos_to_classify_for_Ryan/Nest 028_poor view/Nest 028_poor view/2023_0524_191052_001.JPG: {'predicted_class': 1, 'probabilities': [[0.01427016407251358, 0.9857298135757446]]}
Raw prediction response for Photos_to_classify_for_Ryan/Nest 028_poor view/Nest 028_poor view/2023_0524_191130_001.JPG: {'predicted_class': 1, 'probabilities': [[0.02156793884932995, 0.9784320592880249]]}
Raw prediction response for Photos_to_classify_for_Ryan/Nest 028_poor view/Nest 028_poor view/2023_0524_191401_001.JPG: {'predicted_class': 1, 'probabilities': [[0.025121096521615982, 0.9748789072036743]]}
Raw prediction response for Photos_to_classify_for_Ryan/Nest 028_poor view/Nest 028_poor view/2023_0524_191911_001.JPG: {'predicted_class': 1, 'probabilities': [[0.0120440954342484

...

Raw prediction response for Photos_to_classify_for_Ryan/Nest 029/Nest 029/2023_0525_065703_001.JPG: {'predicted_class': 0, 'probabilities': [[0.9809226393699646, 0.019077379256486893]]}
Image Photos_to_classify_for_Ryan/Nest 029/Nest 029/2023_0525_065703_001.JPG classified as birds with 0.98 probability and added to CSV.
Raw prediction response for Photos_to_classify_for_Ryan/Nest 029/Nest 029/2023_0525_070047_001.JPG: {'predicted_class': 0, 'probabilities': [[0.843337893486023, 0.15666206181049347]]}
Raw prediction response for Photos_to_classify_for_Ryan/Nest 029/Nest 029/2023_0525_070107_001.JPG: {'predicted_class': 0, 'probabilities': [[0.573712170124054, 0.42628785967826843]]}
Raw prediction response for Photos_to_classify_for_Ryan/Nest 029/Nest 029/2023_0525_070125_001.JPG: {'predicted_class': 0, 'probabilities': [[0.6164186000823975, 0.3835814297199249]]}
Raw prediction response for Photos_to_classify_for_Ryan/Nest 029/Nest 029/2023_0525_070142_001.JPG: {'predicted_class': 0, 'p

Raw prediction response for Photos_to_classify_for_Ryan/Nest 029/Nest 029/2023_0525_073621_001.JPG: {'predicted_class': 0, 'probabilities': [[0.9982525706291199, 0.001747367437928915]]}
Image Photos_to_classify_for_Ryan/Nest 029/Nest 029/2023_0525_073621_001.JPG classified as birds with 1.00 probability and added to CSV.
Raw prediction response for Photos_to_classify_for_Ryan/Nest 029/Nest 029/2023_0525_073640_001.JPG: {'predicted_class': 0, 'probabilities': [[0.9952242374420166, 0.004775697831064463]]}
Image Photos_to_classify_for_Ryan/Nest 029/Nest 029/2023_0525_073640_001.JPG classified as birds with 1.00 probability and added to CSV.
Raw prediction response for Photos_to_classify_for_Ryan/Nest 029/Nest 029/2023_0525_073657_001.JPG: {'predicted_class': 0, 'probabilities': [[0.9605294466018677, 0.03947056457400322]]}
Image Photos_to_classify_for_Ryan/Nest 029/Nest 029/2023_0525_073657_001.JPG classified as birds with 0.96 probability and added to CSV.
Raw prediction response for Phot

Raw prediction response for Photos_to_classify_for_Ryan/Nest 029/Nest 029/2023_0525_074944_001.JPG: {'predicted_class': 0, 'probabilities': [[0.9956789612770081, 0.004321024287492037]]}
Image Photos_to_classify_for_Ryan/Nest 029/Nest 029/2023_0525_074944_001.JPG classified as birds with 1.00 probability and added to CSV.
Raw prediction response for Photos_to_classify_for_Ryan/Nest 029/Nest 029/2023_0525_075002_001.JPG: {'predicted_class': 0, 'probabilities': [[0.9956835508346558, 0.004316495265811682]]}
Image Photos_to_classify_for_Ryan/Nest 029/Nest 029/2023_0525_075002_001.JPG classified as birds with 1.00 probability and added to CSV.
Raw prediction response for Photos_to_classify_for_Ryan/Nest 029/Nest 029/2023_0525_075023_001.JPG: {'predicted_class': 0, 'probabilities': [[0.9930249452590942, 0.006975044962018728]]}
Image Photos_to_classify_for_Ryan/Nest 029/Nest 029/2023_0525_075023_001.JPG classified as birds with 0.99 probability and added to CSV.
Raw prediction response for Pho

Raw prediction response for Photos_to_classify_for_Ryan/Nest 029/Nest 029/2023_0525_075922_001.JPG: {'predicted_class': 0, 'probabilities': [[0.999087929725647, 0.0009120734175667167]]}
Image Photos_to_classify_for_Ryan/Nest 029/Nest 029/2023_0525_075922_001.JPG classified as birds with 1.00 probability and added to CSV.
Raw prediction response for Photos_to_classify_for_Ryan/Nest 029/Nest 029/2023_0525_075938_001.JPG: {'predicted_class': 0, 'probabilities': [[0.9913569688796997, 0.008642972446978092]]}
Image Photos_to_classify_for_Ryan/Nest 029/Nest 029/2023_0525_075938_001.JPG classified as birds with 0.99 probability and added to CSV.
Raw prediction response for Photos_to_classify_for_Ryan/Nest 029/Nest 029/2023_0525_075959_001.JPG: {'predicted_class': 0, 'probabilities': [[0.9983214735984802, 0.0016784662147983909]]}
Image Photos_to_classify_for_Ryan/Nest 029/Nest 029/2023_0525_075959_001.JPG classified as birds with 1.00 probability and added to CSV.
Raw prediction response for Ph

Raw prediction response for Photos_to_classify_for_Ryan/Nest 029/Nest 029/2023_0525_081941_001.JPG: {'predicted_class': 1, 'probabilities': [[0.019737044349312782, 0.9802629351615906]]}
Raw prediction response for Photos_to_classify_for_Ryan/Nest 029/Nest 029/2023_0525_081957_001.JPG: {'predicted_class': 1, 'probabilities': [[0.09239280968904495, 0.9076071381568909]]}
Raw prediction response for Photos_to_classify_for_Ryan/Nest 029/Nest 029/2023_0525_082039_001.JPG: {'predicted_class': 1, 'probabilities': [[0.029781460762023926, 0.9702184796333313]]}
Raw prediction response for Photos_to_classify_for_Ryan/Nest 029/Nest 029/2023_0525_082056_001.JPG: {'predicted_class': 1, 'probabilities': [[0.10502605140209198, 0.8949739933013916]]}
Raw prediction response for Photos_to_classify_for_Ryan/Nest 029/Nest 029/2023_0525_082116_001.JPG: {'predicted_class': 1, 'probabilities': [[0.1289149522781372, 0.8710850477218628]]}
Raw prediction response for Photos_to_classify_for_Ryan/Nest 029/Nest 029/

**For moving files**

In [26]:
import boto3

# Initialize the S3 client
s3 = boto3.client('s3')

# Define the S3 path of the text file and target directory for copying images
source_bucket = 'csml-data-bucket'
text_file_key = 'no_birds.txt'
target_bucket = 'csml-data-bucket'
target_prefix = 'preprocessed/preprocessed/unprocessed/'

# Download the text file to the local environment
s3.download_file(source_bucket, text_file_key, '/tmp/no_birds.txt')

# Read the image paths from the downloaded text file
with open('/tmp/no_birds.txt', 'r') as f:
    image_paths = f.readlines()

# Loop through each image path in the text file
for image_path in image_paths:
    image_path = image_path.strip()  # Remove any extra whitespace or newline characters
    if image_path:
        # Extract the source key from the image path (after the bucket name)
        source_key = image_path.replace(f's3://{source_bucket}/', '')

        # Define the target key for the copy operation
        target_key = target_prefix + source_key.split('/')[-1]  # Only keep the image file name

        # Perform the copy operation
        try:
            s3.copy_object(
                Bucket=target_bucket,
                CopySource={'Bucket': source_bucket, 'Key': source_key},
                Key=target_key
            )
            print(f"Successfully copied {source_key} to {target_key}")
        except Exception as e:
            print(f"Failed to copy {source_key}: {e}")


Successfully copied Photos_to_classify_for_Ryan/Nest 064-One GLIB, check again for P2/DCIM/106MEDIA/DSCF0024.JPG to preprocessed/preprocessed/unprocessed/DSCF0024.JPG
Successfully copied Photos_to_classify_for_Ryan/Nest 064-One GLIB, check again for P2/DCIM/106MEDIA/DSCF0025.JPG to preprocessed/preprocessed/unprocessed/DSCF0025.JPG
Successfully copied Photos_to_classify_for_Ryan/Nest 064-One GLIB, check again for P2/DCIM/106MEDIA/DSCF0026.JPG to preprocessed/preprocessed/unprocessed/DSCF0026.JPG
Successfully copied Photos_to_classify_for_Ryan/Nest 064-One GLIB, check again for P2/DCIM/106MEDIA/DSCF0027.JPG to preprocessed/preprocessed/unprocessed/DSCF0027.JPG
Successfully copied Photos_to_classify_for_Ryan/Nest 064-One GLIB, check again for P2/DCIM/106MEDIA/DSCF0025.JPG to preprocessed/preprocessed/unprocessed/DSCF0025.JPG
Successfully copied Photos_to_classify_for_Ryan/Nest 064-One GLIB, check again for P2/DCIM/106MEDIA/DSCF0026.JPG to preprocessed/preprocessed/unprocessed/DSCF0026.JP

Successfully copied Photos_to_classify_for_Ryan/Nest 028_poor view/Nest 028_poor view/2023_0524_163435_001.JPG to preprocessed/preprocessed/unprocessed/2023_0524_163435_001.JPG
Successfully copied Photos_to_classify_for_Ryan/Nest 028_poor view/Nest 028_poor view/2023_0524_163517_001.JPG to preprocessed/preprocessed/unprocessed/2023_0524_163517_001.JPG
Successfully copied Photos_to_classify_for_Ryan/Nest 028_poor view/Nest 028_poor view/2023_0524_163534_001.JPG to preprocessed/preprocessed/unprocessed/2023_0524_163534_001.JPG
Successfully copied Photos_to_classify_for_Ryan/Nest 028_poor view/Nest 028_poor view/2023_0524_163609_001.JPG to preprocessed/preprocessed/unprocessed/2023_0524_163609_001.JPG
Successfully copied Photos_to_classify_for_Ryan/Nest 028_poor view/Nest 028_poor view/2023_0524_163643_001.JPG to preprocessed/preprocessed/unprocessed/2023_0524_163643_001.JPG
Successfully copied Photos_to_classify_for_Ryan/Nest 028_poor view/Nest 028_poor view/2023_0524_163717_001.JPG to p

**To copy the files into a new folder**

In [42]:
import boto3
import csv

# Initialize S3 client
s3_client = boto3.client('s3')

# Define the bucket name and folder paths
source_bucket = 'csml-data-bucket'
destination_bucket = 'csml-data-bucket'
destination_folder_prefix = 'birds_found/'

# Path to the CSV file
csv_file_path = 'birds_left_over.csv'  # Adjust path if necessary

# Function to copy files from source to destination within the same bucket
def copy_files_to_new_folder_from_csv(csv_file_path):
    with open(csv_file_path, mode='r') as file:
        csv_reader = csv.reader(file)
        # Skip the header
        next(csv_reader)
        
        for row in csv_reader:
            # Extract the source file path from the CSV row
            source_key = row[0]  # Assuming the S3 path is the first column
            
            # Define the destination key (path in the new folder structure)
            # Remove the parent folder 'Photos_to_classify_for_Ryan/' from the source path
            relative_path = source_key.replace('Photos_to_classify_for_Ryan/', '')
            destination_key = f"{destination_folder_prefix}{relative_path}"
            
            print(f"Copying {source_key} to {destination_key}")
            
            # Perform the copy operation
            try:
                s3_client.copy_object(
                    Bucket=destination_bucket,
                    CopySource={'Bucket': source_bucket, 'Key': source_key},
                    Key=destination_key
                )
                print(f"Successfully copied {source_key} to {destination_key}")
            except Exception as e:
                print(f"Error copying {source_key} to {destination_key}: {e}")

# Run the copy function
copy_files_to_new_folder_from_csv(csv_file_path)


Copying Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/100MEDIA/DSCF0295.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/100MEDIA/DSCF0295.JPG
Successfully copied Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/100MEDIA/DSCF0295.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/100MEDIA/DSCF0295.JPG
Copying Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/100MEDIA/DSCF0587.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/100MEDIA/DSCF0587.JPG
Successfully copied Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/100MEDIA/DSCF0587.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/100MEDIA/DSCF0587.JPG
Copying Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/100MEDIA/DSCF0588.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/100MEDIA/DSCF0588.JPG
Successfully copied Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for

Successfully copied Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/101MEDIA/DSCF0226.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/101MEDIA/DSCF0226.JPG
Copying Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/101MEDIA/DSCF0227.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/101MEDIA/DSCF0227.JPG
Successfully copied Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/101MEDIA/DSCF0227.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/101MEDIA/DSCF0227.JPG
Copying Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/103MEDIA/DSCF0042.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/103MEDIA/DSCF0042.JPG
Successfully copied Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/103MEDIA/DSCF0042.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/103MEDIA/DSCF0042.JPG
Copying Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for

Successfully copied Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/103MEDIA/DSCF0478.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/103MEDIA/DSCF0478.JPG
Copying Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/103MEDIA/DSCF0479.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/103MEDIA/DSCF0479.JPG
Successfully copied Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/103MEDIA/DSCF0479.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/103MEDIA/DSCF0479.JPG
Copying Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/103MEDIA/DSCF0483.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/103MEDIA/DSCF0483.JPG
Successfully copied Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/103MEDIA/DSCF0483.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/103MEDIA/DSCF0483.JPG
Copying Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for

Successfully copied Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/103MEDIA/DSCF0918.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/103MEDIA/DSCF0918.JPG
Copying Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/103MEDIA/DSCF0919.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/103MEDIA/DSCF0919.JPG
Successfully copied Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/103MEDIA/DSCF0919.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/103MEDIA/DSCF0919.JPG
Copying Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/103MEDIA/DSCF0922.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/103MEDIA/DSCF0922.JPG
Successfully copied Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/103MEDIA/DSCF0922.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/103MEDIA/DSCF0922.JPG
Copying Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for

Successfully copied Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/104MEDIA/DSCF0116.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/104MEDIA/DSCF0116.JPG
Copying Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/104MEDIA/DSCF0117.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/104MEDIA/DSCF0117.JPG
Successfully copied Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/104MEDIA/DSCF0117.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/104MEDIA/DSCF0117.JPG
Copying Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/104MEDIA/DSCF0118.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/104MEDIA/DSCF0118.JPG
Successfully copied Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/104MEDIA/DSCF0118.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/104MEDIA/DSCF0118.JPG
Copying Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for

...

Successfully copied Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/104MEDIA/DSCF0956.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/104MEDIA/DSCF0956.JPG
Copying Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/104MEDIA/DSCF0985.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/104MEDIA/DSCF0985.JPG
Successfully copied Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/104MEDIA/DSCF0985.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/104MEDIA/DSCF0985.JPG
Copying Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/104MEDIA/DSCF0986.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/104MEDIA/DSCF0986.JPG
Successfully copied Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/104MEDIA/DSCF0986.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/104MEDIA/DSCF0986.JPG
Copying Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for

Successfully copied Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/107MEDIA/DSCF0154.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/107MEDIA/DSCF0154.JPG
Copying Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/107MEDIA/DSCF0227.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/107MEDIA/DSCF0227.JPG
Successfully copied Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/107MEDIA/DSCF0227.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/107MEDIA/DSCF0227.JPG
Copying Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/107MEDIA/DSCF0862.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/107MEDIA/DSCF0862.JPG
Successfully copied Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/107MEDIA/DSCF0862.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/107MEDIA/DSCF0862.JPG
Copying Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for

Successfully copied Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/107MEDIA/DSCF0884.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/107MEDIA/DSCF0884.JPG
Copying Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/107MEDIA/DSCF0885.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/107MEDIA/DSCF0885.JPG
Successfully copied Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/107MEDIA/DSCF0885.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/107MEDIA/DSCF0885.JPG
Copying Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/107MEDIA/DSCF0886.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/107MEDIA/DSCF0886.JPG
Successfully copied Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/107MEDIA/DSCF0886.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/107MEDIA/DSCF0886.JPG
Copying Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for

Successfully copied Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/108MEDIA/DSCF0115.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/108MEDIA/DSCF0115.JPG
Copying Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/108MEDIA/DSCF0116.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/108MEDIA/DSCF0116.JPG
Successfully copied Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/108MEDIA/DSCF0116.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/108MEDIA/DSCF0116.JPG
Copying Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/108MEDIA/DSCF0117.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/108MEDIA/DSCF0117.JPG
Successfully copied Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/108MEDIA/DSCF0117.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/108MEDIA/DSCF0117.JPG
Copying Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for

Successfully copied Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/108MEDIA/DSCF0272.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/108MEDIA/DSCF0272.JPG
Copying Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/108MEDIA/DSCF0422.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/108MEDIA/DSCF0422.JPG
Successfully copied Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/108MEDIA/DSCF0422.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/108MEDIA/DSCF0422.JPG
Copying Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/108MEDIA/DSCF0424.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/108MEDIA/DSCF0424.JPG
Successfully copied Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for P2/DCIM/108MEDIA/DSCF0424.JPG to birds_found/Nest 064-One GLIB check again for P2/DCIM/108MEDIA/DSCF0424.JPG
Copying Photos_to_classify_for_Ryan/Nest 064-One GLIB check again for